In [1]:
import os

from transformers import AutoTokenizer
from collections import Counter
from datasets import load_dataset

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

c:\Users\white\anaconda3\envs\HRI-py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
allowed_token_id = set()

In [3]:
# punctuation, whitespace, so on
allowed_token_id = allowed_token_id.union(set(range(0, 256)))

In [4]:
# Special tokens
allowed_token_id = allowed_token_id.union(set(range(128000, 128256)))

In [5]:
# include top_N
import pickle

with open ("top_1000_tokens.pkl", "rb") as f:
    top_N = pickle.load(f)
    allowed_token_id = allowed_token_id.union(set(top_N))

In [6]:
# save the allowed_token_id
with open("allowed_token_id.pkl", "wb") as f:
    pickle.dump(allowed_token_id, f)

### Add target token

In [7]:
import pandas as pd

A1 = pd.read_csv("CEFR/A1.csv")
A2 = pd.read_csv("CEFR/A2.csv")

A1_set = set(A1["word"].values)
A2_set = set(A2["word"].values)

In [8]:
target_base = A1_set.union(A2_set)
capital = set([x.capitalize() for x in target_base])
target_with_punctuation = set(x + p for x in target_base for p in ".,!? \t\n")

In [9]:
target = target_base.union(capital).union(target_with_punctuation)

In [10]:
import os
from transformers import AutoTokenizer

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")


# 토크나이징 후 토큰 ID를 저장할 집합
token_id_set = set()

# 각 요소를 토크나이징하고 토큰 ID를 집합에 추가
for item in target:
    tokens = tokenizer.tokenize(item)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    token_id_set.update(token_ids)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
with open("target_token_id.pkl", "wb") as f:
    pickle.dump(token_id_set, f)

### 통합

In [13]:
final_target = allowed_token_id.union(token_id_set)
with open ("final_target.pkl", "wb") as f:
    pickle.dump(final_target, f)